In [1]:
import networkx as nx
import itertools
from tqdm import tqdm
import pickle

In [2]:
path = '../gs_taxo/EN/environment_eurovoc_en.taxo'
G = nx.DiGraph()

with open(path, 'r') as f:
    for line in f:
        idx, hypo, hyper = line.split('\t')
        hyper = hyper.replace('\n', '')
        G.add_node(hypo)
        G.add_node(hyper)
        G.add_edge(hyper, hypo)

In [3]:
def clean_dict(pairs, use_lemma, reverse):
    new_pairs = {}
    for key, val in pairs.items():
        if use_lemma:
            term = key[0].split("(")[0].strip()
        else:
            term = key[0]
        target = key[1].split(",")[0]
        new_key = (target, term) if reverse else (term, target)
        new_pairs[new_key] = val

    return new_pairs


in_name = '../data/env/eurovoc/pred_hypernym/lemmas_ppl_clear.pickle'
lemma = True
reverse = True

with open(in_name, "rb") as f:
    ppls = pickle.load(f)

ppls_pairs = clean_dict(ppls, use_lemma=lemma, reverse=reverse)

root = 'environment'
all_verteces = list(G.nodes)
all_verteces.remove(root)

In [41]:
def get_graph(ppl_pairs, thr):
    S = nx.DiGraph()
    for key, val in ppl_pairs.items():
        if val <thr:
            S.add_edge(key[0], key[1], weight=val)
    return S
        

G_pred = get_graph(ppls_pairs, 4)

P = len(set(G.edges()) & set(G_pred.edges())) / (len(set(G_pred.edges())) + 1e-15)
R = len(set(G.edges()) & set(G_pred.edges())) / len(set(G.edges()))
# print(len(set(edges)))
F = (2 * P * R) / (P + R + 1e-15)
print(F)

0.2822222222222219


In [33]:
def clean_triplets_conflicts(triplets):
    new_triplets = {}

    for k, v in triplets.items():
        first_parent, second_parent = k[0].split('_')

        new_key = (first_parent, second_parent)
        if new_key in new_triplets.keys():
            new_triplets[new_key][k[1][:-1]] = v
        else:
            new_triplets[new_key] = {k[1][:-1]: v}

    return new_triplets


insertions_path = '../data/env/eurovoc/insertions/lemmas_ppl.pickle'
with open(insertions_path, 'rb') as f:
    insertions = pickle.load(f)

new_ins = clean_triplets_conflicts(insertions)

In [37]:
min(new_ins[('hazardous waste',  'pollution of waterways')].items(), key=lambda x: x[1])

('pollution', 1.3301483392715454)

In [6]:
in_name = '../data/env/eurovoc/leaf_no_leafs/lemmas_ppl_clear.pickle'
lemma = True
reverse = False

with open(in_name, "rb") as f:
    ppls = pickle.load(f)

ppls_hypo = clean_dict(ppls, use_lemma=lemma, reverse=reverse)

In [39]:

def conflict_refinment_ppl(G_pred, ppls_hypo, ppls_pairs, conflict_thr):
    edges = list(G_pred.edges())    
    for u,v in edges:

        diff = abs(ppls_pairs[(u,v)] - ppls_hypo[(u,v)])
pred_hypernym
        if diff > conflict_thr:
           # print(diff)
            G_pred.remove_edge(u,v)

def conflict_refinment_insertion(G_pred,  ppls_hypo, ppls_pairs, conflict_thr, insertions, insert_thr):
    edges = list(G_pred.edges())    
    for u,v in edges:

        diff = abs(ppls_pairs[(u,v)] - ppls_hypo[(u,v)])

        if diff > conflict_thr:
            best_middle_node, best_ppl = min(insertions[(u, v)].items(), key=lambda x: x[1])
            if best_ppl < insert_thr:
                G_pred.add_edge(u, best_middle_node)
                G_pred.add_edge(best_middle_node, v)

    


In [46]:
import numpy as np

np.linspace(1, 55, 40)

array([ 1.        ,  2.38461538,  3.76923077,  5.15384615,  6.53846154,
        7.92307692,  9.30769231, 10.69230769, 12.07692308, 13.46153846,
       14.84615385, 16.23076923, 17.61538462, 19.        , 20.38461538,
       21.76923077, 23.15384615, 24.53846154, 25.92307692, 27.30769231,
       28.69230769, 30.07692308, 31.46153846, 32.84615385, 34.23076923,
       35.61538462, 37.        , 38.38461538, 39.76923077, 41.15384615,
       42.53846154, 43.92307692, 45.30769231, 46.69230769, 48.07692308,
       49.46153846, 50.84615385, 52.23076923, 53.61538462, 55.        ])

In [42]:
conflict_refinment_insertion(G_pred, ppls_hypo=ppls_hypo, ppls_pairs=ppls_pairs, conflict_thr=20, insertions=new_ins, insert_thr=2)

P = len(set(G.edges()) & set(G_pred.edges())) / (len(set(G_pred.edges())) + 1e-15)
R = len(set(G.edges()) & set(G_pred.edges())) / len(set(G.edges()))
# print(len(set(edges)))
F = (2 * P * R) / (P + R + 1e-15)
print(F)

0.2782037239868561
